# 🔧 Utilities

Some utilities for demos, used in other notebooks.

In [ ]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [ ]:
import random

import ipywidgets as W
import traitlets as T

In [ ]:
def make_a_collapsible_picker(title, children):
    ui_children = []
    ui = W.HBox(layout=dict(flex_wrap="wrap"))
    box = W.Accordion([ui], titles=[title])
    select = None
    if len(children) > 1:
        select = W.Dropdown(options=list(children), layout=dict(flex="0"))
        ui.children = [select]
        T.dlink((select, "value"), (ui, "children"), lambda x: [select, *children[x]])
        T.dlink((select, "value"), (box, "titles"), lambda x: (f"{title} ({x})",))
        box.titles = (f"{title} ({select.value})",)
    elif len(children) == 1:
        ui.children = tuple(list(children.values())[0])
    else:
        raise ValueError(f"unexpected number of children {children}")
    box._select = select
    return box

In [ ]:
def make_random_color_series(fg, trait, column_name):
    df = getattr(fg.source, trait)
    digits = "01234567abcdef"
    df[column_name] = [
        "#" + "".join([random.choice(digits) for j in range(6)]) for i in range(len(df))
    ]
    fg.source.send_state(trait)

In [ ]:
def make_link_dropdown_responsive(behavior, label, ui, box):
    box.behaviors = {**box.behaviors, label: None}

    def on_select(change: T.Bunch = None):
        new_behaviors = dict(box.behaviors.items())
        if ui._select.value == "off":
            new_behaviors[label] = None
        else:
            new_behaviors[label] = behavior
        box.behaviors = new_behaviors

    ui._select.observe(on_select, T.All)
    on_select()

In [ ]:
def make_link_behavior_with_ui(WidgetClass, label, column_name, is_color=False):
    def add_behavior(fg, box, column_name=column_name):
        behavior = WidgetClass(column_name=column_name)
        if is_color:
            if column_name not in fg.source.links:
                make_random_color_series(fg, "links", column_name)
        ui_column_name = W.Dropdown(
            options=list(fg.source.links.columns), value=column_name
        )
        ui_template = W.Textarea()
        ui_template_enabled = W.Checkbox(description="enabled?")
        T.link((ui_column_name, "value"), (behavior, "column_name"))
        T.dlink(
            (ui_template, "value"),
            (behavior, "template"),
            lambda x: x if ui_template_enabled.value else "",
        )

        ui = make_a_collapsible_picker(
            label,
            {
                "off": [],
                "column": [ui_column_name],
                "template": [W.VBox([ui_template_enabled, ui_template])],
            },
        )

        box.link_ui = {**box.link_ui, label: ui}

        make_link_dropdown_responsive(behavior, label, ui, box)

        return fg, box

    return add_behavior